**Planning Pattern**

In [2]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

In [3]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>
"""

In [4]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

concatenating the tool signature and add them to the system prompt

In [5]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature


In [6]:
print(tools_signature)


{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"prop

In [7]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature


In [8]:
print(REACT_SYSTEM_PROMPT)



You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

**ReAct Loop Step 1**

In [9]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [10]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

To accomplish the task, we need to follow the order of operations specified: 

1. Calculate the sum of 1234 and 5678.
2. Multiply the result by 5.
3. Compute the logarithm of this result.

First, let's calculate the sum of 1234 and 5678.

<tool_call>
{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 1}
</tool_call>

Next, we'll assume the result of the sum is stored in a variable named "sum_result". We'll multiply this result by 5.

<tool_call>
{"name": "multiply_two_elements", "arguments": {"a": "sum_result", "b": 5}, "id": 2}
</tool_call>

Then, we'll assume the result of the multiplication is stored in a variable named "multiplication_result". We'll compute the logarithm of this result.

<tool_call>
{"name": "compute_log", "arguments": {"x": "multiplication_result"}, "id": 3}
</tool_call>


In [11]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

**Loop 2**

In [12]:
tool_call = extract_tag_content(output, tag="tool_call")


In [13]:
tool_call


TagContentResult(content=['{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 1}', '{"name": "multiply_two_elements", "arguments": {"a": "sum_result", "b": 5}, "id": 2}', '{"name": "compute_log", "arguments": {"x": "multiplication_result"}, "id": 3}'], found=True)

In [14]:
tool_call = json.loads(tool_call.content[0])


In [15]:
tool_call


{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 1}

In [16]:
available_tools[tool_call["name"]]

In [17]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])
tool_result

6912

In [18]:
assert tool_result == 1234 + 5678
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [19]:
chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.\nYou are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\' make assumptions about what values to plug\ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools / actions:\n\n<tools> \n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n\\n    Args:\\n        a (int): The first integer to be summed.\\n        b (int): The second integer to be summed.\\n\\n    Returns:\\n   

**loop 3**

In [20]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

With the result of the sum of 1234 and 5678 being 6912, we can proceed with the next step, which is to multiply this result by 5.

<tool_call>
{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 4}
</tool_call>


In [21]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

**loop 4**

In [22]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [23]:
tool_result


34560

In [24]:
assert tool_result == (1234 + 5678) * 5


In [25]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [26]:
chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.\nYou are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\' make assumptions about what values to plug\ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools / actions:\n\n<tools> \n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n\\n    Args:\\n        a (int): The first integer to be summed.\\n        b (int): The second integer to be summed.\\n\\n    Returns:\\n   

loop 5

In [27]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

With the result of the multiplication being 34560, we can now proceed to compute the logarithm of this result.

<tool_call>
{"name": "compute_log", "arguments": {"x": 34560}, "id": 5}
</tool_call>


In [28]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

ReAct Loop 6

In [29]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [30]:
tool_result


10.450452222917992

In [31]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [32]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

The logarithm of 34560 is approximately 10.450452222917992. This concludes the calculations as per the initial request: calculate the sum of 1234 and 5678, multiply the result by 5, and then take the logarithm of this result. The final result of these operations is 10.450452222917992.


Testing reflection agent library

In [33]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent


In [34]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])


In [35]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")



Thought: []

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 1}

Tool result: 
6912

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 2}

Tool result: 
34560

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 3}

Tool result: 
10.450452222917992

Observations: {1: 6912, 2: 34560, 3: 10.450452222917992}

Thought: []

Thought: []

Thought: []

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 1}

Tool result: 
6912

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 2}

Tool result: 
34560

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 3}

Tool result: 
10.450452222917992

Observations: {1: 6912, 2: 

'The final answer is: $\\boxed{10.450452222917992}$'